# Text Classification Using the Stanford SST Sentiment Dataset
Anne Chen

Github: https://github.com/chenac232/advML



## 1. The dataset
Discuss the dataset in general terms and describe why building a predictive model using this data might be practically useful.  Who could benefit from a model like this? Explain.

The dataset comprises of positive and negative movie reviews. Building a predictive model using this data might be practically useful as it may help better decipher a true movie review score based on the review rather than the assigned score the reviwer gave, which may be eveluated differently from another reviewer. It may also be helpful in evaluating the movie and determining strong positivity or negativity from reviews in other forums.

## Set up

### AI ModelShare

In [1]:
#install aimodelshare library
! pip install aimodelshare==0.0.189

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 967.8/967.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.

In [1]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://rlxjxnoql9.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this specific Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


### Data

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/sst2_competition_data-repository:latest') 


Data downloaded successfully.


In [3]:
# Set up X_train, X_test, and y_train_labels objects
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

X_train=pd.read_csv("sst2_competition_data/X_train.csv", squeeze=True)
X_test=pd.read_csv("sst2_competition_data/X_test.csv", squeeze=True)

y_train_labels=pd.read_csv("sst2_competition_data/y_train_labels.csv", squeeze=True)

# ohe encode Y data
y_train = pd.get_dummies(y_train_labels)

X_train.head()

0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
2    Singer/composer Bryan Adams contributes a slew...
3                 Yet the act is still charming here .
4    Whether or not you 're enlightened by any of D...
Name: text, dtype: object

In [4]:
# This preprocessor function makes use of the tf.keras tokenizer

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 40)
(1821, 40)


## 2. Run at least three prediction models to try to predict the SST sentiment dataset well.
- Use an Embedding layer and LSTM layers in at least one model
- Use an Embedding layer and Conv1d layers in at least one model
- Use transfer learning with glove embeddings for at least one of these models

Discuss which models performed better and point out relevant hyper-parameter values for successful models.
Submit your best three models to the leader board for the SST Model Share competition.

Overall, the transfer learning with glove embedding performed best with an accuracy of 0.98, when compared to the models with an embedding layer and an LSTM or 1D convnets layer. However, it may be that this accuracy boost is simply due to the number of epochs specified in the model. I will be running additional models with varying specifications in the next section to confirm this.

### 2.1 Embedding layer and LSTM layer

In [34]:
from tensorflow.keras.layers import Dense, Embedding,Flatten, SimpleRNN, LSTM
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(10000, 32, input_length=40))
model.add(LSTM(32))
model.add(Dense(2, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(preprocessor(X_train), y_train,
                    epochs=1,
                    batch_size=128,
                    validation_split=0.2)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 40, 32)            320000    
                                                                 
 lstm_9 (LSTM)               (None, 32)                8320      
                                                                 
 dense_16 (Dense)            (None, 2)                 66        
                                                                 
Total params: 328,386
Trainable params: 328,386
Non-trainable params: 0
_________________________________________________________________
44/44 [==============================] - 5s 59ms/step - loss: 0.6538 - acc: 0.6165 - val_loss: 0.9620 - val_acc: 0.1575


#### Model submission

In [35]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [36]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [38]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [39]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 1s 9ms/step
Insert search tags to help users find your model (optional): group5lstm
Provide any useful notes about your model (optional): group5lstm

Your model has been submitted as model version 155

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


### 2.2 Embedding layer with 1D Convnets

In [8]:
# Use 1D Conv layer rather than RNN or LSTM or GRU to fit model
# Why? Much lighter model to fit. Here we are training on the full dataset.  If you try
# to build a model using LSTM code after running this one it will be much slower.

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

model = Sequential()
model.add(layers.Embedding(10000, 32, input_length=40))
model.add(layers.Conv1D(32, 7, activation='relu')) 
model.add(layers.MaxPooling1D(4)) 
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(2))

model.summary()





Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 32)            320000    
                                                                 
 conv1d_4 (Conv1D)           (None, 34, 32)            7200      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 8, 32)            0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 2, 32)             7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 2)                

In [9]:
model.compile(optimizer=RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(preprocessor(X_train), y_train,
                    epochs=1,
                    batch_size=32,
                    validation_split=0.2)

173/173 [==============================] - 2s 7ms/step - loss: 1.9489 - acc: 0.5226 - val_loss: 0.9785 - val_acc: 0.1488


#### Model submission

In [10]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [11]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [12]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [13]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 0s 3ms/step
Insert search tags to help users find your model (optional): conv1dgroup5
Provide any useful notes about your model (optional): conv1d group 5

Your model has been submitted as model version 212

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


### 2.3 Transfer learning with glove embeddings

In [17]:

# Download Glove embedding matrix weights 
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2023-04-17 02:55:28--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-04-17 02:55:28--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-04-17 02:55:29--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [18]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [19]:
import os

# Extract embedding data for 100 feature embedding matrix
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [20]:
maxlen = 40  # We will cut reviews after 40 words in sequence
training_samples = 10000  # We will be training on 10000 samples
validation_samples = 10000  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset


In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [22]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 13835 unique tokens.


In [23]:
# Build embedding matrix
embedding_dim = 100 # change if you use txt files using larger number of features

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [35]:
# Set up same model architecture as before and then import Glove weights to Embedding layer:

import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(10000, embedding_dim, input_length=40))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 100)           1000000   
                                                                 
 flatten_1 (Flatten)         (None, 4000)              0         
                                                                 
 dense_2 (Dense)             (None, 32)                128032    
                                                                 
 dense_3 (Dense)             (None, 2)                 66        
                                                                 
Total params: 1,128,098
Trainable params: 1,128,098
Non-trainable params: 0
_________________________________________________________________


In [36]:

# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False



model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)
model.save_weights('pre_trained_glove_model.h5')


# Training data small to speed up training. Increase for better fit.


Epoch 1/10
173/173 [==============================] - 2s 7ms/step - loss: 0.6196 - acc: 0.6512 - val_loss: 0.7396 - val_acc: 0.5643
Epoch 2/10
173/173 [==============================] - 1s 6ms/step - loss: 0.4993 - acc: 0.7543 - val_loss: 0.8180 - val_acc: 0.5470
Epoch 3/10
173/173 [==============================] - 1s 6ms/step - loss: 0.4217 - acc: 0.8040 - val_loss: 0.5215 - val_acc: 0.7666
Epoch 4/10
173/173 [==============================] - 1s 6ms/step - loss: 0.3542 - acc: 0.8463 - val_loss: 0.7291 - val_acc: 0.6821
Epoch 5/10
173/173 [==============================] - 1s 7ms/step - loss: 0.2884 - acc: 0.8802 - val_loss: 1.0665 - val_acc: 0.5571
Epoch 6/10
173/173 [==============================] - 1s 6ms/step - loss: 0.2268 - acc: 0.9187 - val_loss: 0.9810 - val_acc: 0.6199
Epoch 7/10
173/173 [==============================] - 1s 7ms/step - loss: 0.1739 - acc: 0.9417 - val_loss: 0.9506 - val_acc: 0.6402
Epoch 8/10
173/173 [==============================] - 1s 6ms/step - loss: 0.

#### Model submission

In [37]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [38]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model3.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [39]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [40]:
#Submit Model 3: 

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model3.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 0s 2ms/step
Insert search tags to help users find your model (optional): glove1
Provide any useful notes about your model (optional): glove1

Your model has been submitted as model version 265

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


## 3. Three more models after team discussion
After you submit your first three models, describe your best model with your team via your team slack channel
Fit and submit up to three more models after learning from your team.
Discuss results

**Team Discussion:** In the team discussion, we found that increasing the number of epochs for each model specification yielded to the greatest boost in accuracy score.

**Updated model results:**
- For the model with the LSTM layer, when I increase the epoch = 10, the accuracy score improves to nearly 0.8 (from a score of 0.5 with epoch=1).
- Increasing epochs for the 1D convnets model did not yield a similar boost in accuracy score, however, it stayed around 0.5-0.6.
- For the glove embeddings model, instead of increasing epochs I increased the number of units in the dense layer from 32 to 64. This yielded minimal improvements from the initial model in part 2.

### Model 1 - LSTM + embedding layers

In [14]:
from tensorflow.keras.layers import Dense, Embedding,Flatten, SimpleRNN, LSTM
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(10000, 32, input_length=40))
model.add(LSTM(32))
model.add(Dense(2, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 40, 32)            320000    
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 328,386
Trainable params: 328,386
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
44/44 [==============================] - 4s 43ms/step - loss: 0.6490 - acc: 0.6172 - val_loss: 0.7548 - val_acc: 0.4328
Epoch 2/10
44/44 [==============================] - 1s 31ms/step - loss: 0.5140 - acc: 0.7558 - val_loss: 0.6850 - val_acc: 0.5860
Epoch 3/10
44/44 [==============================] 

#### Model submission

In [15]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [16]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model4.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [17]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [18]:
#Submit Model 3: 

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model4.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 1s 7ms/step
Insert search tags to help users find your model (optional): lstm model 2
Provide any useful notes about your model (optional): lstm model 2

Your model has been submitted as model version 218

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


### Model 2 - 1D conv + embedding layers

In [49]:
# Use 1D Conv layer rather than RNN or LSTM or GRU to fit model
# Why? Much lighter model to fit. Here we are training on the full dataset.  If you try
# to build a model using LSTM code after running this one it will be much slower.

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

model = Sequential()
model.add(layers.Embedding(10000, 32, input_length=40))
model.add(layers.Conv1D(32, 7, activation='relu')) 
model.add(layers.MaxPooling1D(4)) 
model.add(layers.Conv1D(64, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(2))

model.summary()





Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 40, 32)            320000    
                                                                 
 conv1d_17 (Conv1D)          (None, 34, 32)            7200      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 8, 32)            0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 2, 64)             14400     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_9 (Dense)             (None, 2)               

In [50]:
model.compile(optimizer=RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 3s 10ms/step - loss: 0.9629 - acc: 0.6142 - val_loss: 0.8863 - val_acc: 0.1488
Epoch 2/10
173/173 [==============================] - 1s 7ms/step - loss: 0.6653 - acc: 0.6149 - val_loss: 0.8720 - val_acc: 0.1488
Epoch 3/10
173/173 [==============================] - 1s 6ms/step - loss: 0.6613 - acc: 0.6149 - val_loss: 0.8890 - val_acc: 0.1488
Epoch 4/10
173/173 [==============================] - 1s 7ms/step - loss: 0.6567 - acc: 0.6149 - val_loss: 0.9329 - val_acc: 0.1488
Epoch 5/10
173/173 [==============================] - 1s 6ms/step - loss: 0.6511 - acc: 0.6149 - val_loss: 0.8416 - val_acc: 0.1488
Epoch 6/10
173/173 [==============================] - 1s 6ms/step - loss: 0.6432 - acc: 0.6149 - val_loss: 0.8859 - val_acc: 0.1488
Epoch 7/10
173/173 [==============================] - 1s 6ms/step - loss: 0.6312 - acc: 0.6149 - val_loss: 0.8941 - val_acc: 0.1488
Epoch 8/10
173/173 [==============================] - 1s 6ms/step - loss: 0

#### Model submission

In [51]:
import aimodelshare as ai


In [52]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model5.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [53]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [54]:
#Submit Model 3: 

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model5.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 0s 3ms/step
Insert search tags to help users find your model (optional): convnets
Provide any useful notes about your model (optional): convnets

Your model has been submitted as model version 251

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


### Model 3 - transfer learning with glove embeddings

In [24]:
# Build embedding matrix
embedding_dim = 100 # change if you use txt files using larger number of features

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [25]:
# Set up same model architecture as before and then import Glove weights to Embedding layer:

import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(10000, embedding_dim, input_length=40))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 100)           1000000   
                                                                 
 flatten (Flatten)           (None, 4000)              0         
                                                                 
 dense (Dense)               (None, 64)                256064    
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1,256,194
Trainable params: 1,256,194
Non-trainable params: 0
_________________________________________________________________


In [26]:

# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False



model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)
model.save_weights('pre_trained_glove_model.h5')


# Training data small to speed up training. Increase for better fit.


Epoch 1/10
173/173 [==============================] - 2s 6ms/step - loss: 0.6272 - acc: 0.6393 - val_loss: 0.6874 - val_acc: 0.6257
Epoch 2/10
173/173 [==============================] - 1s 5ms/step - loss: 0.4934 - acc: 0.7561 - val_loss: 0.6634 - val_acc: 0.6763
Epoch 3/10
173/173 [==============================] - 1s 5ms/step - loss: 0.4019 - acc: 0.8177 - val_loss: 0.8755 - val_acc: 0.5882
Epoch 4/10
173/173 [==============================] - 1s 5ms/step - loss: 0.3143 - acc: 0.8748 - val_loss: 0.7217 - val_acc: 0.6821
Epoch 5/10
173/173 [==============================] - 1s 7ms/step - loss: 0.2299 - acc: 0.9133 - val_loss: 0.9233 - val_acc: 0.6416
Epoch 6/10
173/173 [==============================] - 1s 7ms/step - loss: 0.1591 - acc: 0.9532 - val_loss: 0.8680 - val_acc: 0.6785
Epoch 7/10
173/173 [==============================] - 1s 8ms/step - loss: 0.1076 - acc: 0.9702 - val_loss: 0.9019 - val_acc: 0.6944
Epoch 8/10
173/173 [==============================] - 1s 8ms/step - loss: 0.

#### Model submission

In [31]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [32]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model5.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [33]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [34]:
#Submit Model 3: 

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model5.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 0s 2ms/step
Insert search tags to help users find your model (optional): glove embedding model 2
Provide any useful notes about your model (optional): glove embedding model 2

Your model has been submitted as model version 264

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


## 4. Which model performed best? 
Discuss which models you tried and which models performed better and point out relevant hyper-parameter values for successful models.

Overall, the LSTM and embeddings layer with epochs =10 yielded the greatest amount of model accuracy, while the convnets model had the poorest initial accuracy. Increasing epochs in the convnets model did not boost model accuracy to the same magnitude when the LSTM model was boosted. The glove embeddings model performed well initially with epochs=10. Adding the number of units in the dense layer from 32 to 64 did not amount to much of a model accuracy boost. Ultimately, the number of epochs was a great determinant in improving model accuracy scores.